In [ ]:
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
include("../src/utils.jl")
include("../src/unit_commitments.jl")
include("../src/network_analysis.jl")
include("../src/plots.jl")
ENV["COLUMNS"]=120;

gen_df, gen_var_long, loads_long, network = process_data("../WECC")

zone_dict = Dict(zip(gen_df.zone, gen_df.zone_name));
line_dict = Dict(zip(network.network_lines, network."transmission path name"));

In [ ]:
# A spring day
n=100
initial_time = n*24+6
final_time = n*24+30
T_period = (initial_time):(final_time)

loads_multi = loads_long[in.(loads_long.hour,Ref(T_period)),:]
gen_var_multi = gen_var_long[in.(gen_var_long.hour, Ref(T_period)), :]

uc_solution = uc_with_transport(
    gen_df, loads_multi, gen_var_multi, network, 0.01); #1% MIP Gap

In [ ]:
gen_vre = gen_df[gen_df[!,:vre] .== 1, :r_id]
mask = in.(ruc_gen_var_multi.r_id, Ref(gen_vre))
ruc_gen_var_multi[mask, :cf] .*= 1.5 # it's real sunny ! and windy !

times = 1:22
ruc_solutions = Dict{Int, Any}()
for recommit_time in times
    T_period = (initial_time + recommit_time):(final_time)

    loads_multi = loads_long[in.(loads_long.hour,Ref(T_period)),:]
    ruc_gen_var_multi = gen_var_long[in.(gen_var_long.hour, Ref(T_period)), :]

    ruc_solution = reliability_uc(
        solution.decision_vars, initial_time + recommit_time,
        gen_df, loads_multi, ruc_gen_var_multi, network, 0.01); #1% MIP Gap
    ruc_solutions[recommit_time] = ruc_solution
end
